In [ ]:
#hide 
#Standard Lib imports
import sys, os, shutil
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 
%config Completer.use_jedi = False

In [ ]:
from rfpye.parser import parse_bin, export_metadata, extract_level
from rfpye.main import extract_bin_data
from rfpye.utils import get_files, df_optimize
import numpy as np
import pandas as pd
from fastcore.foundation import L
from rich import print

In [ ]:
entrada = Path('/content/data')
saida = Path('/content/pq')
saida.mkdir(exist_ok=True)

In [ ]:
arquivos = get_files('/content/bin', extensions=['.bin']) ;  arquivos

In [ ]:
arquivo = arquivos[-1]

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import json

In [ ]:
# for p in arquivos[-10:]:
#     shutil.copyfile(p, f'/content/{p.name}')

In [ ]:
%%time
dados = extract_bin_data(arquivo, save_path=saida, spec_data=True)

In [ ]:
out = dados['rfeye002239_210618_T174320.bin']

In [ ]:
out['Fluxos']

In [ ]:
fluxo = out['Fluxos'][67,100]

In [ ]:
level = fluxo['Level_Data']

In [ ]:
level.shape

(2576, 1024)

In [ ]:
level_flat = level.flatten() ; level_flat.shape

(2637824,)

In [ ]:
(level_flat.reshape(fluxo['Num_Traces'], -1) == level).all()

True

In [ ]:
header_key = 'rfeye002239_210618_T174320.bin'
header_meta_json = json.dumps({k:str(v) for k,v in out.items() if k!= 'Fluxos'})
combined_meta = {header_key.encode(): header_meta_json.encode()}

for (btype, tid), fluxo in out['Fluxos'].items():
    combined_meta = {header_key.encode(): header_meta_json.encode()}
    meta_key = f'{btype}-{tid}_meta'    
    table = pa.table({'Level_Data': fluxo['Level_Data'].flatten()}, nthreads=os.cpu_count())
                    
#         'Minimum_Level': fluxo['Minimum_Level'],
#                       'Timestamp': fluxo['Timestamp']}, )
    
    existing_meta = table.schema.metadata
    custom_meta = {k:str(v) for k,v in fluxo.items() if k not in ('Timestamp', 'Minimum_Level', 'Level_Data')}
    
    custom_meta_json = json.dumps(custom_meta)
    custom_meta = {meta_key.encode() : custom_meta_json.encode()}
    combined_meta.update(custom_meta)
    table = table.replace_schema_metadata(combined_meta)
    pq.write_table(table, f'{saida}/{header_key}_{btype}-{tid}.parquet', compression='GZIP') 

In [ ]:
restored_table = pq.read_table(f'{saida}/{header_key}_67-300.parquet')

In [ ]:
restored_table.schema.metadata

{b'67-300_meta': b'{"Initial_Time": "2021-06-18T17:47:03.155018000", "Sample_Duration": "11662.0", "Description": "PMEC 2021 (Faixa 1 de 10).", "Start_Frequency": "105", "Stop_Frequency": "140", "Trace_Type": "peak", "RBW": "18457", "Level_Units": "dBm", "Num_Traces": "4271", "Vector_Length": "3584", "Minimum_Level": "[-147.5 -147.5 -147.5 ... -147.5 -147.5 -147.5]"}',
 b'rfeye002239_210618_T174320.bin': b'{"File_Name": "rfeye002239_210618_T174320.bin", "File_Version": "23", "File_Type": "RFEye Logger Trace", "Device": "Rfeye Node", "Equipment_ID": "rfeye002239", "Script_Version": "ScriptRFeye2021_v2.cfg", "Latitude": "-23.442419052124023", "Longitude": "-46.4725227355957", "Altitude": "807.7999877929688", "Count_GPS": "4433", "Sum_Latitude": "-103920.25", "Sum_Longitude": "-206012.69"}'}

In [ ]:
restored_table[b'67-300']

In [ ]:
restored_df = restored_table.to_pandas()

In [ ]:
restored_df.head()

,Level_Data
0,123
1,127
2,134
3,121
4,142


In [ ]:
from math import pi
from bokeh.io import show, output_notebook
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure

In [ ]:
data = pd.DataFrame(levels, index=timestamp, columns=frequencies)
data.columns.name = "Frequencies"
data.index.name = "Time"
data.head()

In [ ]:
df = pd.DataFrame(data.stack(), columns=['Levels']).reset_index() ; df.head()

In [ ]:
frequencies

In [ ]:
time = timestamp.tolist()
frequencies = frequencies.astype('float32').tolist()

In [ ]:
# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.Levels.min(), high=df.Levels.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Waterfall ({0} - {1})".format(timestamp[0], timestamp[-1]),
           x_range=frequencies, y_range=timestamp,
           x_axis_label = 'Frequencies',
           y_axis_label = 'Timestamp',
           x_axis_location="above", plot_width=1920, plot_height=1080,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('Frequência', '@Frequencies'), ('Tempo', '@Time'), ('Nível', '@Levels')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "0px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="Frequencies", 
       y="Time", 
       width=1,
       height=1, 
       source=df,
       fill_color={'field': 'Levels', 'transform': mapper}, 
       line_color=None)



color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10px",
                 ticker=BasicTicker(desired_num_ticks=len(colors)),
                 formatter=PrintfTickFormatter(format="%ddBm"),
                 label_standoff=6, border_line_color=None)

p.add_layout(color_bar, 'right')

show(p)      # show the plot